In [1]:
# To ignore warnings
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn import metrics

%matplotlib inline

In [3]:
df_football_GK_Defenders_position = pd.read_csv(r"C:\Users\foooo\Datascincecamp\Usecase-7\df_football_GK_Defenders_position_cleaned.csv")

In [5]:
df_football_GK_Defenders_position.shape

(5629, 20)

In [7]:
df_football_GK_Defenders_position.head(2)

,Unnamed: 0,team,position,height,age,appearance,goals,assists,yellow cards,second yellow cards,red cards,goals conceded,clean sheets,minutes played,days_injured,games_injured,award,current_value,highest_value,position_encoded
0,0,Manchester United,Goalkeeper,189.0,32,104,0.0,0.0,0.009585,0.0,0.0,1.217252,0.335463,9390,42,5,13,15000000,70000000,1
1,1,Manchester United,Goalkeeper,196.0,30,15,0.0,0.0,0.069018,0.0,0.0,1.242331,0.207055,1304,510,58,1,1500000,22000000,1


In [11]:
df_football_GK_Defenders_position.columns

Index(['Unnamed: 0', 'team', 'position', 'height', 'age', 'appearance',
       'goals', 'assists', 'yellow cards', 'second yellow cards', 'red cards',
       'goals conceded', 'clean sheets', 'minutes played', 'days_injured',
       'games_injured', 'award', 'current_value', 'highest_value',
       'position_encoded'],
      dtype='object')

In [15]:
df_football_GK_Defenders_position.drop('Unnamed: 0',axis=1, inplace= True)

In [19]:
df_football_GK_Defenders_position.dtypes

team                    object
position                object
height                 float64
age                      int64
appearance               int64
goals                  float64
assists                float64
yellow cards           float64
second yellow cards    float64
red cards              float64
goals conceded         float64
clean sheets           float64
minutes played           int64
days_injured             int64
games_injured            int64
award                    int64
current_value            int64
highest_value            int64
position_encoded         int64
dtype: object

In [21]:
categorical_features = ['team' ,'position']
numeric_features = ['position', 'height', 'age', 'appearance',
       'goals', 'assists', 'yellow cards', 'second yellow cards', 'red cards',
       'goals conceded', 'clean sheets', 'minutes played', 'days_injured',
       'games_injured', 'award', 'current_value', 'highest_value',
       'position_encoded']

In [23]:
df_football_GK_Defenders_position = pd.get_dummies(df_football_GK_Defenders_position, columns=categorical_features)

In [25]:
df_football_GK_Defenders_position.head(1)

,height,age,appearance,goals,assists,yellow cards,second yellow cards,red cards,goals conceded,clean sheets,...,team_Yokohama F. Marinos,team_Yokohama FC,team_Zenit St. Petersburg,team_Ümraniyespor,position_Defender,position_Defender Centre-Back,position_Defender Left-Back,position_Defender Right-Back,position_Goalkeeper,position_midfield-DefensiveMidfield
0,189.0,32,104,0.0,0.0,0.009585,0.0,0.0,1.217252,0.335463,...,False,False,False,False,False,False,False,False,True,False


In [27]:
correlation = df_football_GK_Defenders_position.corr()
print(correlation['current_value'].sort_values(ascending=False))

current_value                  1.000000
highest_value                  0.847484
minutes played                 0.435470
appearance                     0.435064
award                          0.261865
                                 ...   
team_Jeonbuk Hyundai Motors   -0.024655
position_Defender             -0.030689
age                           -0.062845
goals conceded                -0.078861
position_Goalkeeper           -0.099745
Name: current_value, Length: 397, dtype: float64


In [31]:
threshold = 0.4  # You can change this value based on your requirement

# Filter the correlations
# We use `abs()` for absolute value to consider both strong positive and negative correlations
selected_features = correlation[abs(correlation['current_value']) > threshold]['current_value'].index
selected_features

Index(['appearance', 'minutes played', 'current_value', 'highest_value'], dtype='object')

In [33]:
selected_features = ['appearance', 'minutes played', 'current_value', 'highest_value']

In [37]:
df_football_GK_Defenders_position = df_football_GK_Defenders_position[selected_features]
df_football_GK_Defenders_position.head()

,appearance,minutes played,current_value,highest_value
0,104,9390,15000000,70000000
1,15,1304,1500000,22000000
2,4,292,600000,6000000
3,82,6408,50000000,50000000
4,63,5031,40000000,80000000


In [39]:
X = df_football_GK_Defenders_position.drop(['current_value'], axis=1)
y = df_football_GK_Defenders_position['current_value']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    shuffle=True,
                                                    random_state=42)

# sacle the data
scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [41]:
model = LinearRegression()

In [43]:
model.fit(X_train_scaled, y_train)

LinearRegression()

In [45]:
y_pred = model.predict(X_test_scaled)

In [47]:
model.coef_

array([-642606.54732267, 1298764.26307753, 5893512.83455238])

In [49]:
coeff_df = pd.DataFrame(model.coef_,X.columns,columns=['Coefficient'])
coeff_df

,Coefficient
appearance,-6.426065e+05
minutes played,1.298764e+06
highest_value,5.893513e+06


In [51]:
print(model.intercept_) 

2942388.407728181
